In [2]:
!python -V

Python 3.11.11


# Convert docx to txt

In [ ]:
import os
from preprocess import convert_doc_to_txt

directories = ['HienPhap', 'LuatLaoDong', 'LuatDanSu', 'LuatHinhSu']
processed_files = []
failed_files = []

for directory in directories:
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.doc', '.docx')) and not file.startswith('~$'):
                file_path = os.path.join(root, file)
                if file_path.split(".")[0] in processed_files:
                    continue
                try:
                    # Process the file
                    print(f"Processing {file_path}...")
                    convert_doc_to_txt(file_path)
                    print(f"Successfully processed {file_path}.")
                    processed_files.append(file_path.split(".")[0])
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    failed_files.append(file_path.split(".")[0])

for file in failed_files:
    print(f"Failed: {file}")

# Preprocess

In [30]:
!python preprocess.py

2025-04-13 14:50:22,131 - INFO - Đang xử lý thư mục HienPhap...
2025-04-13 14:50:22,145 - INFO - Đang xử lý file: HienPhap\hien-phap-2013.doc
2025-04-13 14:50:22,225 - INFO - Đã xử lý thành công file: HienPhap\hien-phap-2013.doc
2025-04-13 14:50:22,225 - INFO - Tổng số file đã xử lý của thư mục HienPhap: 1
2025-04-13 14:50:22,225 - INFO - Tổng số file thất bại của thư mục HienPhap: 0
2025-04-13 14:50:22,225 - INFO - Đang xử lý thư mục LuatLaoDong...
2025-04-13 14:50:22,226 - INFO - Đang xử lý file: LuatLaoDong\bo-luat-lao-dong-2019.doc
2025-04-13 14:50:22,368 - INFO - Đã xử lý thành công file: LuatLaoDong\bo-luat-lao-dong-2019.doc
2025-04-13 14:50:22,368 - INFO - Đang xử lý file: LuatLaoDong\luat-an-toan-ve-sinh-lao-dong-2015.doc
2025-04-13 14:50:22,431 - INFO - Đã xử lý thành công file: LuatLaoDong\luat-an-toan-ve-sinh-lao-dong-2015.doc
2025-04-13 14:50:22,432 - INFO - Đang xử lý file: LuatLaoDong\luat-cong-doan-2012.doc
2025-04-13 14:50:22,453 - INFO - Đã xử lý thành công file: LuatL

# Add law_text and metadata index

In [37]:
import json

# file_path = 'HienPhap/hien-phap-2013.json'
# file_path = 'LuatLaoDong/bo-luat-lao-dong-2019.json'
# file_path = 'LuatDanSu/bo-luat-dan-su-2015.json'
# file_path = 'LuatHinhSu/bo-luat-hinh-su-2015.json'
# file_path = 'LuatLaoDong/van-ban-hop-nhat-19.json'
file_path = 'LuatHinhSu/van-ban-hop-nhat-01.json'
with open(file_path, 'r') as f:
    data = json.load(f)
print(data.keys())

dict_keys(['law_name', 'metadata', 'parts'])


In [ ]:
from preprocess import iterate_law_recursive, get_law_text
import numpy as np

i = 0
length = []
num_words = []
num_length_greater_than_512 = 0
with open('result.txt', 'w') as f:
    for item in iterate_law_recursive(data):
        law_text = get_law_text(item)
        f.write(law_text + '\n\n')
        length.append(len(law_text))
        num_words.append(len(law_text.split()))
        i += 1
        if num_words[-1] > 512:
            num_length_greater_than_512 += 1

print("length")
print(np.average(length))
print(np.min(length))
print(np.max(length))
print(num_length_greater_than_512, i)
print("num_words")
print(np.average(num_words))
print(np.min(num_words))
print(np.max(num_words))

length
390.68916712030483
121
1459
0 3674
num_words
86.25748502994011
28
304


In [ ]:
# result_path = 'hien-phap-0-306.json'
# result_path = 'lao-dong-0-911.json'
# result_path = 'dan-su-0-1923.json'
# result_path = 'hinh-su-0-3741.json'
# result_path = 'van-ban-hop-nhat-19-0-597.json'
result_path = 'van-ban-hop-nhat-01-0-3673.json'
with open(result_path, 'r') as f:
    result = json.load(f)
print(result[0].keys())

dict_keys(['law_summary', 'metadata', 'qas'])


In [6]:
from preprocess import iterate_law_recursive
import re

i = 0
for item in iterate_law_recursive(data):
    i += 1
print(i)
print(len(result))

3674
3674


In [ ]:
from preprocess import iterate_law_recursive, get_law_text
import re

i = 0
for item in iterate_law_recursive(data):
    
    law_text = get_law_text(item)

    result[i]['metadata']['index'] = {
        'law_name': item['law'],
        'part': item['part'][0],
        'chapter': item['chapter'][0],
        'section': item['section'][0],
        'subsection': item['subsection'][0],
        'article': item['article'][0],
        'item': item['item'][0],
    }
    result[i] = {
        'law_text': law_text,
        'law_summary': result[i]['law_summary'],
        'metadata': result[i]['metadata'],
        'qas': result[i]['qas'],
    }
    i += 1

In [8]:
with open(result_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# Add full article

In [ ]:
import json

# file_path = 'HienPhap/hien-phap-2013.json'
# file_path = 'LuatLaoDong/bo-luat-lao-dong-2019.json'
# file_path = 'LuatDanSu/bo-luat-dan-su-2015.json'
# file_path = 'LuatHinhSu/bo-luat-hinh-su-2015.json'
# file_path = 'LuatLaoDong/van-ban-hop-nhat-19.json'
file_path = 'LuatHinhSu/van-ban-hop-nhat-01.json'
with open(file_path, 'r') as f:
    data = json.load(f)
print(data.keys())

dict_keys(['law_name', 'metadata', 'chapters'])


In [ ]:
# result_path = 'hien-phap-0-306.json'
# result_path = 'lao-dong-0-911.json'
# result_path = 'dan-su-0-1923.json'
# result_path = 'hinh-su-0-3741.json'
# result_path = 'van-ban-hop-nhat-19-0-597.json'
result_path = 'van-ban-hop-nhat-01-0-3673.json'
with open(result_path, 'r') as f:
    result = json.load(f)
print(result[0].keys())

dict_keys(['full_law', 'law_text', 'law_summary', 'metadata', 'qas'])


In [4]:
from preprocess import iterate_law_recursive
import re

i = 0
for item in iterate_law_recursive(data):
    i += 1
print(i)
print(len(result))

912
912


In [7]:
def append_new_from_str2(str1, str2):
    str1 = str1.splitlines()
    str2 = str2.splitlines()
    
    for i, item in enumerate(str2):
        if item not in str1:
            # Từ phần tử này đến cuối, append vào a
            str1.extend(str2[i:])
            break
    return '\n'.join(str1)

current_law_text = ''
current_index_article = -1
num_item_in_article = 0
list_article = []
for item in iterate_law_recursive(data):
    law_text = get_law_text(item)
    if item['article'][0] == current_index_article:
        current_law_text = append_new_from_str2(current_law_text, law_text)
        num_item_in_article += 1
    else:
        if num_item_in_article:
            # list_article += [get_only_article_content(current_law_text)] * num_item_in_article
            list_article += [current_law_text] * num_item_in_article
        current_index_article = item['article'][0]
        current_law_text = law_text
        num_item_in_article = 1
list_article += [current_law_text] * num_item_in_article

In [ ]:
# with open('test.json', 'w', encoding='utf-8') as f:
#     json.dump(list_article, f, ensure_ascii=False, indent=4)

In [ ]:
# import numpy as np

# length = []
# num_words = []
# num_length_greater_than_512 = 0
# for article in list_article:
#     length.append(len(article))
#     num_words.append(len(article.split()))
#     if num_words[-1] > 512:
#         num_length_greater_than_512 += 1

# print("length")
# print(np.average(length))
# print(np.min(length))
# print(np.max(length))
# print(num_length_greater_than_512, len(list_article))
# print("num_words")
# print(np.average(num_words))
# print(np.min(num_words))
# print(np.max(num_words))

length
1939.4077299945563
144
11061
852 3674
num_words
418.6861731083288
31
2379


In [30]:
for i in range(len(result)):
    result[i] = {
        'full_law': list_article[i],
        # 'law_text': get_only_article_content(result[i]['law_text']),
        'law_text': result[i]['law_text'],
        'law_summary': result[i]['law_summary'],
        'metadata': result[i]['metadata'],
        'qas': result[i]['qas'],
    }

In [31]:
with open(result_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# Export csv

In [40]:
import json
import csv
import re

def get_only_article_content(law_text):
    try:
        match = re.search(r'\n(Đi.{1,2}u\s+\d+.*)', law_text, re.DOTALL)
        return match.group(1)
    except Exception as e:
        print(law_text)
        print(e)

def process_json_to_csv(json_path, questions_path, corpus_path, qnc_path, law_code='hs'):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    question_rows = []
    corpus_rows = []
    qnc_rows = []

    question_counter = 1
    corpus_counter = 1

    corpus_map = {}  # để tránh trùng lặp nội dung corpus

    for entry in data:
        # --- xử lý corpus ---
        full_law = get_only_article_content(entry.get('full_law', '').strip())
        law_text = get_only_article_content(entry.get('law_text', '').strip())
        law_summary = entry.get('law_summary', '').strip()

        # helper để thêm content và sinh id nếu chưa có
        def add_corpus(content):
            nonlocal corpus_counter
            if content and content not in corpus_map:
                cid = f'{law_code}c{corpus_counter:06d}'
                corpus_rows.append([cid, content])
                corpus_map[content] = cid
                corpus_counter += 1
            return corpus_map.get(content)

        # add unique corpus content
        cid_text = add_corpus(law_text)
        cid_summary = add_corpus(law_summary)
        if full_law and full_law != law_text:
            cid_full = add_corpus(full_law)
        else:
            cid_full = None

        # --- xử lý câu hỏi ---
        for qa in entry.get('qas', []):
            qid = f'{law_code}q{question_counter:06d}'
            question_rows.append([qid, qa['question'].strip()])
            question_counter += 1

            # nối câu hỏi với các corpus id tương ứng
            if cid_text:
                qnc_rows.append([qid, cid_text])
            if cid_summary:
                qnc_rows.append([qid, cid_summary])
            if cid_full:
                qnc_rows.append([qid, cid_full])

    # Ghi ra 3 file csv
    with open(questions_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['question_id', 'question'])
        writer.writerows(question_rows)

    with open(corpus_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['corpus_id', 'content'])
        writer.writerows(corpus_rows)

    with open(qnc_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['question_id', 'corpus_id'])
        writer.writerows(qnc_rows)

In [51]:
# Ví dụ sử dụng
process_json_to_csv(
    json_path='hinh-su-sua-doi-0-3673.json',
    questions_path='hinh-su-questions.csv',
    corpus_path='hinh-su-corpus.csv',
    qnc_path='hinh-su-qnc.csv',
    law_code='hs'
)

In [6]:
import pandas as pd

df = pd.read_csv('dan-su-corpus.csv')
df['content'][0]

'BỘ LUẬT DÂN SỰ\nPhần thứ nhất QUY ĐỊNH CHUNG\nChương I NHỮNG QUY ĐỊNH CHUNG\nĐiều 1 Phạm vi điều chỉnh\nBộ luật này quy định địa vị pháp lý, chuẩn mực pháp lý về cách ứng xử của cá nhân, pháp nhân; quyền, nghĩa vụ về nhân thân và tài sản của cá nhân, pháp nhân trong các quan hệ được hình thành trên cơ sở bình đẳng, tự do ý chí, độc lập về tài sản và tự chịu trách nhiệm (sau đây gọi chung là quan hệ dân sự).'

# Get all docs

In [2]:
law_mapping = {
    "HIẾN PHÁP NƯỚC CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM": "Hiến pháp 2013",
    "BỘ LUẬT DÂN SỰ": "Bộ luật dân sự 2015",
    "LUẬT BẢO VỆ QUYỀN LỢI NGƯỜI TIÊU DÙNG": "Luật Bảo vệ quyền lợi người tiêu dùng 2023",
    "LUẬT HÔN NHÂN VÀ GIA ĐÌNH": "Luật Hôn nhân và gia đình 2014",
    "BỘ LUẬT HÌNH SỰ": "Bộ luật Hình sự sửa đổi 2017",
    "BỘ LUẬT LAO ĐỘNG": "Bộ luật Lao động 2019",
    "LUẬT AN TOÀN, VỆ SINH LAO ĐỘNG": "Luật an toàn, vệ sinh lao động 2015",
    "LUẬT CÔNG ĐOÀN": "Luật Công đoàn 2012",
    "LUẬT VIỆC LÀM": "Luật việc làm 2013",
    "LUẬT BẢO HIỂM XÃ HỘI": "Luật Bảo hiểm xã hội sửa đổi 2019"
}

In [3]:
from preprocess import iterate_law_recursive, get_law_text
import json

def append_new_from_str2(str1, str2):
    str1 = str1.splitlines()
    str2 = str2.splitlines()
    
    for i, item in enumerate(str2):
        if item not in str1:
            # Từ phần tử này đến cuối, append vào a
            str1.extend(str2[i:])
            break
    return '\n'.join(str1)

file_path = [
    'HienPhap/hien-phap-2013.json',
    'LuatDanSu/bo-luat-dan-su-2015.json',
    'LuatDanSu/luat-bao-ve-quyen-loi-nguoi-tieu-dung-2023.json',
    'LuatDanSu/luat-hon-nhan-va-gia-dinh-2014.json',
    'LuatHinhSu/van-ban-hop-nhat-01.json',
    'LuatLaoDong/bo-luat-lao-dong-2019.json',
    'LuatLaoDong/luat-an-toan-ve-sinh-lao-dong-2015.json',
    'LuatLaoDong/luat-cong-doan-2012.json',
    'LuatLaoDong/luat-viec-lam-2013.json',
    'LuatLaoDong/van-ban-hop-nhat-19.json'
]
result_path = [
    'hien-phap-0-306.json',
    'dan-su-0-1923.json',
    'luat-bao-ve-quyen-loi-nguoi-tieu-dung-2023-0-511.json',
    'luat-hon-nhan-va-gia-dinh-2014-0-402.json',
    'hinh-su-sua-doi-0-3673.json',
    'lao-dong-0-911.json',
    'luat-an-toan-ve-sinh-lao-dong-2015-0-519.json',
    'luat-cong-doan-2012-0-120.json',
    'luat-viec-lam-2013-0-255.json',
    'luat-bao-hiem-xa-hoi-0-597.json'
]

all_docs = []
for f, r in zip(file_path, result_path):
    print(f)
    print(r)
    with open(f, 'r') as f:
        data = json.load(f)
    with open(r, 'r') as f:
        result = json.load(f)

    current_law_text = ''
    current_index_article = -1
    num_item_in_article = 0
    list_article = []
    for item in iterate_law_recursive(data):
        law_text = get_law_text(item, law_mapping)
        if item['article'][0] == current_index_article:
            current_law_text = append_new_from_str2(current_law_text, law_text)
            num_item_in_article += 1
        else:
            if num_item_in_article:
                # list_article += [get_only_article_content(current_law_text)] * num_item_in_article
                list_article += [current_law_text]
            current_index_article = item['article'][0]
            current_law_text = law_text
            num_item_in_article = 1
    list_article += [current_law_text]
    all_docs += list_article

HienPhap/hien-phap-2013.json
hien-phap-0-306.json
LuatDanSu/bo-luat-dan-su-2015.json
dan-su-0-1923.json
LuatDanSu/luat-bao-ve-quyen-loi-nguoi-tieu-dung-2023.json
luat-bao-ve-quyen-loi-nguoi-tieu-dung-2023-0-511.json
LuatDanSu/luat-hon-nhan-va-gia-dinh-2014.json
luat-hon-nhan-va-gia-dinh-2014-0-402.json
LuatHinhSu/van-ban-hop-nhat-01.json
hinh-su-sua-doi-0-3673.json
LuatLaoDong/bo-luat-lao-dong-2019.json
lao-dong-0-911.json
LuatLaoDong/luat-an-toan-ve-sinh-lao-dong-2015.json
luat-an-toan-ve-sinh-lao-dong-2015-0-519.json
LuatLaoDong/luat-cong-doan-2012.json
luat-cong-doan-2012-0-120.json
LuatLaoDong/luat-viec-lam-2013.json
luat-viec-lam-2013-0-255.json
LuatLaoDong/van-ban-hop-nhat-19.json
luat-bao-hiem-xa-hoi-0-597.json


In [4]:
len(all_docs)

1979

In [5]:
with open('all_docs.json', 'w', encoding='utf-8') as f:
    json.dump(all_docs, f, ensure_ascii=False, indent=4)

In [18]:
all_docs[945]

'LUẬT HÔN NHÂN VÀ GIA ĐÌNH\nChương IV CHẤM DỨT HÔN NHÂN\nMục 1 LY HÔN\nĐiều 59 Nguyên tắc giải quyết tài sản của vợ chồng khi ly hôn\nTrong trường hợp chế độ tài sản của vợ chồng theo luật định thì việc giải quyết tài sản do các bên thỏa thuận; nếu không thỏa thuận được thì theo yêu cầu của vợ, chồng hoặc của hai vợ chồng, Tòa án giải quyết theo quy định tại các khoản 2, 3, 4 và 5 Điều này và tại các điều 60, 61, 62, 63 và 64 của Luật này. Trong trường hợp chế độ tài sản của vợ chồng theo thỏa thuận thì việc giải quyết tài sản khi ly hôn được áp dụng theo thỏa thuận đó; nếu thỏa thuận không đầy đủ, rõ ràng thì áp dụng quy định tương ứng tại các khoản 2, 3, 4 và 5 Điều này và tại các điều 60, 61, 62, 63 và 64 của Luật này để giải quyết.\nTài sản chung của vợ chồng được chia đôi nhưng có tính đến các yếu tố sau đây:\nHoàn cảnh của gia đình và của vợ, chồng;\nCông sức đóng góp của vợ, chồng vào việc tạo lập, duy trì và phát triển khối tài sản chung. Lao động của vợ, chồng trong gia đình đ